The implementation of Algorithm GNN, presented on page 19, equipped with selective power method. The results of the corresponding numerical experiments are given on pages 36 and 37.

In [ ]:
#Block 1: loading packages
import numpy as np
from numpy.linalg import norm
import time
from scipy.sparse import rand

In [ ]:
#Block 2: function which generates a random integer from 0 to k-1
rndm = lambda k: np.random.randint(k)

In [ ]:
#Block 3: setting the rounding and precision parameters
prec = 8 #rounding parameter
Eps = 10**(-prec)

In [ ]:
#Block 4: selective power method 
def pwrmthd(A):
    A = A + np.identity(dim) #see the remark on page 28
    v0 = np.array([1 for i in xrange(dim)]) #starting vector of all ones 
    v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    v1 = np.round(v1,prec)
    while norm(v0-v1) > Eps*10: #the precision parametar $\varepsilon$ 
        v0 = v1
        v1 = np.dot(A,v0)/float(norm(np.dot(A,v0)))
    return np.round(v1,prec-1)

In [ ]:
#Block 5: function for computing the spectral radius
def leading(A):
    
    evals = np.linalg.eig(A)[0] #set of eigenvalues 
    return np.amax(np.real(evals)) #spectral radius

In [ ]:
#Block 6: generating the positive uncertainty set
def uncertainty_poz():
    return [np.random.rand(dim,) for i in xrange(N)]

In [ ]:
#Block 7: generating the sparse uncertainty set
def uncertainty_sparse(N):
    ranrow = lambda: (rand(1,dim,density = np.random.uniform(a,b))).A
    l = []
    for i in xrange(N):
        v = ranrow()
        v = v.flatten()
        l.append(v)
    return l

In [ ]:
#Block 8: selective greedy method for maximization
def selective_greedyMAX(A):
    
    Z = np.copy(A)
    
    count = 0 #the iteration counter
    start = time.clock() #time counter

    while True:
        
        v = pwrmthd(Z)
        count += 1
        change = False
        
        for i in xrange(dim):
            
            l = np.dot(Z[i],v)
            l = np.round(l,prec)
        
            for x in F[i]: #finding the vec. from the prod. set which maximizes the dot prod.
                prod = np.dot(x,v)
                prod = np.round(prod,prec)
                
                if (prod > l):# and (abs(prod - l) >= \delta); see the Appendix
                    # change (prod < l) for minimization
                    l = prod
                    Z[i] = x
                    change = True
            
        if not change: #if no row was changed, we finish the procedure
            break
                
    
    run = np.round(time.clock() - start,2)
    spect_radius = np.round(leading(Z),2)
    
    return spect_radius, count, run, (v > 0).all()
    '''the last item checks if the final leading eigenvcetor is strictly positive; 
    omit it for minimization'''
    

In [ ]:
#Block 9: running the code, for sparse case
dim = 1000 #setting the dimension
N = 100 #setting the cardinality of the uncertainty sets
(a,b) = (0.09,0.16) #setting the interval from which density parameter take value
F = [uncertainty_sparse(N) for i in xrange(dim)] #generating the product family F
A = np.array([F[i][rndm(N)] for i in xrange(dim)]) #taking an arbitrary matrix from F
G = selective_greedyMAX(A) #running the selective greedy method
print G